In [1]:
# Import necessary packages
from dotenv import load_dotenv
import os
import pandas as pd
import pickle
import faiss
import textwrap

# Import LangChain and OpenAI components
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import PromptTemplate

# Import our custom components
from src.rag.retriever import CustomRetriever, RerankingRetriever
from src.chain.qa_chain import build_qa_chain

# Load environment variables
load_dotenv()

# Set working directory
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')

# Load data and models
df = pd.read_parquet(os.getenv('PARQUET_PATH'))
faiss_index = faiss.read_index(os.getenv('FAISS_INDEX_PATH'))
with open(os.getenv('ID_MAPPING_PATH'), 'rb') as f:
    id_mapping = pickle.load(f)

# Initialize embeddings model
embeddings = OpenAIEmbeddings(openai_api_key=os.getenv('OPENAI_API_KEY'))

In [2]:
# 1. Initialize the base retriever
custom_retriever = CustomRetriever(
    embeddings=embeddings,
    faiss_index=faiss_index,
    id_mapping=id_mapping,
    documents_df=df,
    k=20
)

# 2. Load the reranker prompt
with open('src/prompts/reranker_prompt.txt', 'r', encoding='utf-8') as f:
    reranker_prompt_template = f.read()
reranker_prompt = PromptTemplate.from_template(reranker_prompt_template)

# 3. Initialize the reranker LLM
reranker_llm = ChatOpenAI(
    model='gpt-4o-mini',
    temperature=0,
    openai_api_key=os.getenv('OPENAI_API_KEY')
)

# 4. Initialize the RerankingRetriever
reranking_retriever = RerankingRetriever(
    retriever=custom_retriever,
    llm=reranker_llm,
    reranker_prompt=reranker_prompt,
    embeddings=embeddings,
    k=5 # The reranker will return the top 5 documents
)

In [3]:
# 5. Build the final Question-Answering chain
qa_chain = build_qa_chain(retriever=reranking_retriever)

# 6. Define the query and invoke the chain
query = "Mely volt az utóbbi évek legfontosabb Kúriai döntése?"
result = qa_chain.invoke(query)

# 7. Print the final answer
print(f"❓ Query: {query}\n")
print("✅ Final Answer:\n")
print(textwrap.fill(result, width=100))


--> [Retriever] Starting document retrieval process...
--> [Retriever] Searching FAISS index...
--> [Retriever] Found 20 potential chunks from FAISS. Fetching from DB.
--> [Retriever] Fetching data from LOCAL PARQUET file...
--> [Retriever] Found 0 documents in Parquet file.
--> [Retriever] No documents found in DB for the retrieved IDs.
❓ Query: Mely volt az utóbbi évek legfontosabb Kúriai döntése?

✅ Final Answer:

1. Szintetizált Válasz:   A megadott dokumentumok alapján a kérdés nem válaszolható meg.  2.
Részletes Elemzés:   * A kérdés megválaszolásához szükséges információ, miszerint mely volt az
utóbbi évek legfontosabb Kúriai döntése, nem található meg a rendelkezésre bocsátott
dokumentumokban.  3. Következtetés:   A dokumentumok nem tartalmaznak információt az utóbbi évek
legfontosabb Kúriai döntéséről.  4. Jogi nyilatkozat:   Ez az elemzés kizárólag a rendelkezésre
bocsátott dokumentumrészletek alapján készült, és nem minősül jogi tanácsadásnak.
